In [1]:
import pandas as pd 
import json
import os
import yaml
from pathlib import Path
from typing import Dict   # pour typer les dictionnaires dans les fonctions


In [2]:
def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Goal:
        Function to load configuration parameters from a YAML file.
    Parameters:
        config_filename (str): Relative or absolute path to the YAML file.
    Returns:
        dict: A dictionary containing the configuration parameters.
    """

    config_path = Path(config_filename)

    # Si le chemin est relatif, on le résout depuis la racine du projet
    if not config_path.is_absolute():
        project_root = Path(__file__).resolve().parents[1]
        config_path = project_root / config_path

    if not config_path.exists():
        raise FileNotFoundError(f"Config file not found: {config_path}")

    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config

def fct_read_csv(root_file: str) -> pd.DataFrame:
    """
    Goal:
        Function to read a CSV file and return a pandas DataFrame.
    Parameters:
        root_file (str): Relative or absolute path to the CSV file.
    Returns:
        pd.DataFrame: The DataFrame containing the data from the CSV file.
    """
    seps = [',', ';', '|', '\t']

    file_path = Path(root_file)

    if not file_path.is_absolute():
        try:
            # Cas script Python
            project_root = Path(__file__).resolve().parents[1]
        except NameError:
            # Cas notebook Jupyter
            project_root = Path.cwd().parent

        file_path = project_root / file_path

    if not file_path.exists():
        print(f"Erreur : fichier {file_path} introuvable")
        return pd.DataFrame()

    for sep in seps:
        try:
            df = pd.read_csv(
                file_path,
                sep=sep,
                encoding="utf-8",
                skipinitialspace=True
            )

            # Si plus d'une colonne → bon séparateur
            if df.shape[1] > 1:
                return df

        except Exception:
            continue

    print(f"Aucun séparateur valide trouvé pour {file_path}")
    return pd.DataFrame()


def fct_read_json_nested(root_file: str) -> dict:
    """
    Goal
        Function to read a JSON file and return a pandas DataFrame.
    Parameters:
        root_file (str): The path to the JSON file.
    Returns:
        pd.DataFrame: The DataFrame containing the data from the JSON file.
    """
    with open(root_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    dfs = {}

    # --------------------
    # DIMENSIONS
    # --------------------
    dfs['stadiums'] = pd.DataFrame(data.get('stadiums', []))
    dfs['tvchannels'] = pd.DataFrame(data.get('tvchannels', []))
    dfs['teams'] = pd.DataFrame(data.get('teams', []))

    # --------------------
    # MATCHES - GROUP STAGE
    # --------------------
    matches = []

    for group_key, group_data in data.get('groups', {}).items():
        for match in group_data.get('matches', []):
            match_flat = match.copy()
            match_flat['group'] = group_key
            match_flat['stage'] = 'group'
            matches.append(match_flat)

    # --------------------
    # MATCHES - KNOCKOUT
    # --------------------
    for round_key, round_data in data.get('knockout', {}).items():
        for match in round_data.get('matches', []):
            match_flat = match.copy()
            match_flat['group'] = None
            match_flat['stage'] = round_key
            matches.append(match_flat)

    dfs['matches'] = pd.DataFrame(matches)

    return dfs



def fct_add_prefix_to_df(df:pd.DataFrame, prefix:str) -> pd.DataFrame:
    """
    Goal:
        Function to add a prefix to all column names in a DataFrame.
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        prefix (str): The prefix to add to each column name.
    Returns:
        pd.DataFrame: The DataFrame with updated column names.
    """
    for col in df.columns:
        df.rename(columns={col: f"{prefix}_{col}"}, inplace=True)
    return df



def fct_extract_data(
    root_csv_2010: str,
    root_csv_2014: str,
    root_csv_2022: str,
    root_json_2018: str
) -> None:
    """
    Goal:
        Function to extract data from a CSV files and JSON file to a consolidated DataFrame df.
    Parameters:
        root_csv_2010 (str): The path to the first CSV file.
        root_csv_2014 (str): The path to the second CSV file.
        root_csv_2022 (str): The path to the third CSV file.
        root_json_2018 (str): The path to the JSON file.
    Returns:
        pd.DataFrame: The consolidated DataFrame containing data from all files.
    """
    df_2010 = fct_read_csv(root_csv_2010)
    df_2014 = fct_read_csv(root_csv_2014)
    df_2022 = fct_read_csv(root_csv_2022)
    df_2018 = fct_read_json_nested(root_json_2018)
    
    print(df_2010.head())
    print(df_2014.head())
    print(df_2022.head())
    print(df_2018['matches'].head())
    # df = pd.concat([df_2010, df_2014, df_2022, df_2018], ignore_index=True)
    return None


In [3]:
# Load configuration parameters from config.yaml
config_path = os.path.join(Path.cwd().parent, 'config.yaml')
config = fct_load_config(config_path)

root_csv_2022 = config['root_csv_2022']

# read data in dfs
df_2022 = fct_read_csv(root_csv_2022)


In [4]:
df_2022

,team1,team2,possession team1,possession team2,possession in contest,number of goals team1,number of goals team2,date,hour,category,...,penalties scored team1,penalties scored team2,goal preventions team1,goal preventions team2,own goals team1,own goals team2,forced turnovers team1,forced turnovers team2,defensive pressures applied team1,defensive pressures applied team2
0,QATAR,ECUADOR,42%,50%,8%,0,2,20 NOV 2022,17 : 00,Group A,...,0,1,6,5,0,0,52,72,256,279
1,ENGLAND,IRAN,72%,19%,9%,6,2,21 NOV 2022,14 : 00,Group B,...,0,1,8,13,0,0,63,72,139,416
2,SENEGAL,NETHERLANDS,44%,45%,11%,0,2,21 NOV 2022,17 : 00,Group A,...,0,0,9,15,0,0,63,73,263,251
3,UNITED STATES,WALES,51%,39%,10%,1,1,21 NOV 2022,20 : 00,Group B,...,0,1,7,7,0,0,81,72,242,292
4,ARGENTINA,SAUDI ARABIA,64%,24%,12%,1,2,22 NOV 2022,11 : 00,Group C,...,1,0,4,14,0,0,65,80,163,361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,ENGLAND,FRANCE,54%,36%,10%,1,2,10 DEC 2022,20 : 00,Quarter-final,...,1,0,9,15,0,0,49,54,193,308
60,ARGENTINA,CROATIA,34%,54%,12%,3,0,13 DEC 2022,20 : 00,Semi-final,...,1,0,12,10,0,0,85,63,321,260
61,FRANCE,MOROCCO,34%,55%,11%,2,0,14 DEC 2022,20 : 00,Semi-final,...,0,0,13,14,0,0,72,47,328,218
62,CROATIA,MOROCCO,45%,45%,10%,2,1,17 DEC 2022,16 : 00,Play-off for third place,...,0,0,7,14,0,0,75,72,288,277


In [5]:
list_wanted_columns = [
    'team1', 
    'team2',
    'number of goals team1', 
    'number of goals team2', 
    'date',
    'hour',
    'category'
]
df_2022_filtered = df_2022[list_wanted_columns].copy()
df_2022_filtered.head()
    

,team1,team2,number of goals team1,number of goals team2,date,hour,category
0,QATAR,ECUADOR,0,2,20 NOV 2022,17 : 00,Group A
1,ENGLAND,IRAN,6,2,21 NOV 2022,14 : 00,Group B
2,SENEGAL,NETHERLANDS,0,2,21 NOV 2022,17 : 00,Group A
3,UNITED STATES,WALES,1,1,21 NOV 2022,20 : 00,Group B
4,ARGENTINA,SAUDI ARABIA,1,2,22 NOV 2022,11 : 00,Group C


In [6]:
df = df_2022_filtered.copy()
df = df.rename(columns={
    "team1": "home_team",
    "team2": "away_team",
    "number of goals team1": "home_result",
    "number of goals team2": "away_result",
    "category": "stage",
})

In [7]:
# nettoyer l'heure "17 : 00" -> "17:00"
df["hour"] = df["hour"].astype("string").str.replace(" ", "", regex=False)

# parse date + hour (mois en anglais)
dt = pd.to_datetime(
    df["date"].astype("string").str.strip() + " " + df["hour"].astype("string"),
    errors="coerce"
)

df["date"] = dt.dt.strftime("%Y%m%d%H%M%S")
df = df.drop(columns=["hour"])

/tmp/ipykernel_432213/4042952051.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(


In [8]:
#Noms des équipes avec la première lettre en majuscule
df["home_team"] = df["home_team"].astype("string").str.strip().str.lower().str.title()
df["away_team"] = df["away_team"].astype("string").str.strip().str.lower().str.title()

In [9]:
# Résultats en int 
df["home_result"] = pd.to_numeric(df["home_result"], errors="coerce").astype("Int64")
df["away_result"] = pd.to_numeric(df["away_result"], errors="coerce").astype("Int64")

In [10]:
df

,home_team,away_team,home_result,away_result,date,stage
0,Qatar,Ecuador,0,2,20221120170000,Group A
1,England,Iran,6,2,20221121140000,Group B
2,Senegal,Netherlands,0,2,20221121170000,Group A
3,United States,Wales,1,1,20221121200000,Group B
4,Argentina,Saudi Arabia,1,2,20221122110000,Group C
...,...,...,...,...,...,...
59,England,France,1,2,20221210200000,Quarter-final
60,Argentina,Croatia,3,0,20221213200000,Semi-final
61,France,Morocco,2,0,20221214200000,Semi-final
62,Croatia,Morocco,2,1,20221217160000,Play-off for third place


### Harmoniser colonne stage

In [11]:
df['stage'].unique()

array(['Group A', 'Group B', 'Group C', 'Group D', 'Group F', 'Group E',
       'Group G', 'Group H', 'Round of 16', 'Quarter-final', 'Semi-final',
       'Play-off for third place', 'Final'], dtype=object)

In [12]:
# Harminser la colonne 'stage' avec les valeurs définies dans le fichier config.yaml
mapping_dict = config['stage_mapping_2022']
df["stage"] = df["stage"].replace(mapping_dict)

In [13]:
df['stage'].unique()

array(['group_a', 'group_b', 'group_c', 'group_d', 'group_f', 'group_e',
       'group_g', 'group_h', 'round_of_16', 'quarter_finals',
       'semi_finals', 'third_place', 'final'], dtype=object)

In [14]:
# ajout colonne edition 
df['edition'] = df['date'].str[:4].astype(int)

In [15]:

def transform_2022_data(df: pd.DataFrame , config: dict) -> pd.DataFrame:
    """
    Transforme les données brutes des matchs 2022 en un format nettoyé et standardisé.

    Les opérations réalisées sont :
    - Sélection et renommage des colonnes utiles
    - Nettoyage du format de l'heure
    - Fusion de la date et de l'heure en un timestamp unique (YYYYMMDDHHMMSS)
    - Normalisation des noms d'équipes (Title Case)
    - Conversion des scores en entiers (nullable Int64)

    Parameters
    ----------
    raw_df : pandas.DataFrame
        DataFrame contenant les données brutes des matchs avec au minimum
        les colonnes suivantes :
        - 'team1'
        - 'team2'
        - 'number of goals team1'
        - 'number of goals team2'
        - 'date'
        - 'hour'
        - 'category'

    Returns
    -------
    pandas.DataFrame
        DataFrame transformé avec les colonnes :
        - home_team (str)
        - away_team (str)
        - home_result (Int64)
        - away_result (Int64)
        - date (str, format YYYYMMDDHHMMSS)
        - stage (str)

    Notes
    -----
    - Les dates invalides ou mal formées sont converties en NaT puis en NaN.
    - Les scores non numériques sont convertis en valeurs manquantes (pd.NA).
    """
    
    list_wanted_columns = [
    'team1', 
    'team2',
    'number of goals team1', 
    'number of goals team2', 
    'date',
    'hour',
    'category'
    ]

    # Filtrage des colonnes
    df_filtered = df[list_wanted_columns].copy()

    df_filtered = df_filtered.rename(columns={
    "team1": "home_team",
    "team2": "away_team",
    "number of goals team1": "home_result",
    "number of goals team2": "away_result",
    "category": "stage",
    })

    # nettoyer l'heure "17 : 00" -> "17:00"
    df_filtered["hour"] = df_filtered["hour"].astype("string").str.replace(" ", "", regex=False)

    # parse date + hour (mois en anglais)
    # dt = pd.to_datetime(
    # df_filtered["date"].astype("string").str.strip() + " " + df_filtered["hour"].astype("string"),
    # errors="coerce"
    # )
    dt = pd.to_datetime(
        df_filtered["date"].astype(str).str.strip() + " " + df_filtered["hour"].astype(str).str.strip(),
        format="%d %b %Y %H:%M",  # exemple : '01 Jan 2022 15:30'
        errors="coerce",
        dayfirst=True  # si ton format est jour/mois/année
    )

    df_filtered["date"] = dt.dt.strftime("%Y%m%d%H%M%S")
    df_filtered = df_filtered.drop(columns=["hour"])

    #Noms des équipes avec la première lettre en majuscule
    df_filtered["home_team"] = df_filtered["home_team"].astype("string").str.strip().str.lower().str.title()
    df_filtered["away_team"] = df_filtered["away_team"].astype("string").str.strip().str.lower().str.title()

    # Résultats en int 
    df_filtered["home_result"] = pd.to_numeric(df_filtered["home_result"], errors="coerce").astype("Int64")
    df_filtered["away_result"] = pd.to_numeric(df_filtered["away_result"], errors="coerce").astype("Int64")
    
    # Harminser la colonne 'stage' avec les valeurs définies dans le fichier config.yaml
    mapping_dict = config['stage_mapping_2022']
    df_filtered["stage"] = df_filtered["stage"].replace(mapping_dict)
    
    # Trier par date (ascendant : plus ancien en premier)
    df_filtered.sort_values("date", inplace=True)
    # Réinitialiser l'index incrémental pour match_id
    df_filtered["match_id"] = range(1, len(df_filtered) + 1)
    # ajout colonne edition 
    df_filtered['edition'] = df_filtered['date'].str[:4].astype(int)
    df_filtered["city"] = None
    
    # Réorganisation des colonnes du DataFrame
    df_filtered = df_filtered[["match_id", "date", "home_team", "away_team", "home_result", "away_result", "stage", "edition", "city"]]


    
    return df_filtered